In [3]:
import numpy as np
import math 
from scipy import spatial
import pickle
from collections import Counter

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
with open('/content/drive/MyDrive/NLP_proj/words.pkl', 'rb') as f:
    words = pickle.load(f)

with open('/content/drive/MyDrive/NLP_proj/words_vectors.pkl', 'rb') as f:
    x = np.asarray(pickle.load(f))

In [12]:
n = len(words) 
d = np.zeros((n,n))
d_current = np.zeros((n,n))

# calculate distance matrix

In [ ]:
for i in range (0,n):
    if i % 1000 == 0:
        print(f'{i}/{n}')
    for j in range (i,n):
        d[i][j] = spatial.distance.cosine(x[i], x[j])
        d[j][i] = d[i][j]

# find keys

In [ ]:
def detect_key(d_current , c_current):
    m = d_current.shape[0]
    temp = []
    k = []
    s = []
    n = 0
#############   find I1    ############# min mean

    for i in range (0,m):# i1
        sum = 0
        for j in range (0,m):
            sum += d_current[i][j]

        temp.append(sum / m) 

    I1 = temp.index(sorted(temp)[0])  


    for i in range (0,m):                       # update s and k for I1
        if i != I1:
            k.append(i)
    s.append(I1)

########################################



    while( n < c_current - 1 ):
        min_i = []


#############   find In    ############# max min

        for i in k:                                 # find min 
            min = float("-infinity")
            for j in s:
                if d_current[i][j] < min:
                    min = d_current[i][j]
            min_i.append(min)                       # list of min values in each row

        p = 0
        for i in k:                                 # find max of mins
            for j in s:
                if d_current[i][j] == max( min_i):
                    i_n = i
                    k.remove(i_n)
                    p = 1
                    break
            if p == 1:
                break

        s.append(i_n)
        n+=1
    return s

     



# main code

In [ ]:
def key_clustering( dd , c_target, k1 , g ):


########################################### initial

    n = dd.shape[0]
    k = k1 + 1
    end_key = 2
    list_k = []
    r = np.zeros((n,k))

########################################### calculate R

    for i in range (0,n):
        list_k = sorted(dd[i])[0:k]                          # if K = 2 take 2 min value k = 2 
        for j in range(0,n):                                 # find index of that values
            for x in range(0,len(list_k)):
                if dd[i][j] == list_k[x] :
                        r[i][x] = j

                                         
    l = list(range(n))
    c_previous = n
    c_current = math.floor(n/g)
    s_current = []
    
    ########################### calculate d_current

    for i in range (0,n):
        for j in range(0,n):
            temp = 0
            for a in range (0,k):
                for b in range (0,k):
                    temp = temp + dd[int(r[i][a] ),int(r[j][b])]
            d_current[i][j] = temp / ((k)*(k))
    
    ############## loop #############

    while c_current > c_target :
            s_current = detect_key(d_current ,c_current)

            ############## update labels ##############
            
            for i in range(len(l)):
                    min = float("-infinity")                      # initialize with infinity
                    if(l[i] not in s_current):
                        for j in s_current:
                            if d_current[l[i],j] < min:
                                min = d_current[ l[i] , j ]
                                index = j
                        l[i] = index
    
            #################  update distance ###############

            unique_values =list( Counter(l).keys())             # equals to list(set(words))
            unique_count = list(Counter(l).values())            # counts the elements' frequency
           
            sample = []
            for i in unique_count:
                e = np.zeros((i*(k)))
                sample.append(e)
            
            p = []
            
            for x in range(0,len(unique_values)):
                nn = 0
                for y in range(0,len(l)):
                    if unique_values[x] == l[y]:
                        sample[x][nn] = y
                        nn = nn+1

                        for i in range(1,k):
                            sample[x][nn] = r[y][i] 
                            nn = nn+1

            new_temp = []
            for e in sample:
                new_temp.append(list(map(int, e))) 
            
            label_dict = dict(zip(unique_values, new_temp))

            for x,y  in label_dict.items():
                for p,q in label_dict.items():
                    sum  = 0
                    for a in y:
                        for b in q:
                            sum+= dd[a,b]

                    d_current[x,p] = sum /(len(q)*len(y))

            c_previous = c_current
            c_current = math.floor(c_current/g)

    s_final = detect_key(d_current ,c_target)

    ############## update labels ##############
    for i in range(len(l)):
        min = float("-infinity")
        if(l[i] not in s_final):
            for j in s_final:
                if d_current[l[i],j] < min:
                    min = d_current[l[i],j]
                    index = j
            l[i] = index

    return l

In [ ]:
dd = np.copy(d)
c_target = 5
k1 = 2
g = 3
key_clustering(dd, c_target, k1 , g)